# Indicators of Heart Disease 

In [52]:
import os
import pymysql
import sys
import mysql.connector
from sqlalchemy import create_engine
import numpy 
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json 

In [53]:
#reading code in as csv
health_data = pd.read_csv("C:/Users/Student/Desktop/UVA/UVA '22 Spring/DS systems/heart_2020_cleaned.csv")

In [54]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "tgomtsyan"
pwd = "Password1535"

db_name = "heart"

In [55]:
#converting csv to json 

# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            key = rows['HeartDisease']
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
# Driver Code
 
# Decide the two file paths according to your
# computer system
csvFilePath = "C:/Users/Student/Desktop/UVA/UVA '22 Spring/DS systems/heart_2020_cleaned.csv"
jsonFilePath = "C:/Users/Student/Desktop/UVA/UVA '22 Spring/DS systems/heart_2020_cleaned.json"
 
# Call the make_json function
make_json(csvFilePath, jsonFilePath)

In [56]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [61]:
#renaming, changind, and deleting columns in dataframe

# Renaming column with confusing name
dfnew = health_data.rename(columns={'DiffWalking': 'Difficulty Walking'})
dfnew

#dropping physicalhealth column, because the numbers are not interpretable 
del dfnew['PhysicalHealth']
dfnew

#using drop function now to delete another column 
dfdrop = dfnew.drop(['MentalHealth'], axis=1)
dfdrop

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,Difficulty Walking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
5,Yes,28.87,Yes,No,No,Yes,Female,75-79,Black,No,No,Fair,12.0,No,No,No
6,No,21.63,No,No,No,No,Female,70-74,White,No,Yes,Fair,4.0,Yes,No,Yes
7,No,31.64,Yes,No,No,Yes,Female,80 or older,White,Yes,No,Good,9.0,Yes,No,No
8,No,26.45,No,No,No,No,Female,80 or older,White,"No, borderline diabetes",No,Fair,5.0,No,Yes,No
9,No,40.69,No,No,No,Yes,Male,65-69,White,No,Yes,Good,10.0,No,No,No


In [64]:
#rearranging columns to make more sense to me 

clist = ['Sex','AgeCategory','Race','BMI', 'Smoking', 'AlcoholDrinking','PhysicalActivity', 'SleepTime', 'Difficulty Walking', 'HeartDisease','Diabetic', 'Asthma', 'KidneyDisease', 'SkinCancer', 'GenHealth']
dfnew = dfnew[clist]
dfnew

,Sex,AgeCategory,Race,BMI,Smoking,AlcoholDrinking,PhysicalActivity,SleepTime,Difficulty Walking,HeartDisease,Diabetic,Asthma,KidneyDisease,SkinCancer,GenHealth
0,Female,55-59,White,16.60,Yes,No,Yes,5.0,No,No,Yes,Yes,No,Yes,Very good
1,Female,80 or older,White,20.34,No,No,Yes,7.0,No,No,No,No,No,No,Very good
2,Male,65-69,White,26.58,Yes,No,Yes,8.0,No,No,Yes,Yes,No,No,Fair
3,Female,75-79,White,24.21,No,No,No,6.0,No,No,No,No,No,Yes,Good
4,Female,40-44,White,23.71,No,No,Yes,8.0,Yes,No,No,No,No,No,Very good
5,Female,75-79,Black,28.87,Yes,No,No,12.0,Yes,Yes,No,No,No,No,Fair
6,Female,70-74,White,21.63,No,No,Yes,4.0,No,No,No,Yes,No,Yes,Fair
7,Female,80 or older,White,31.64,Yes,No,No,9.0,Yes,No,Yes,Yes,No,No,Good
8,Female,80 or older,White,26.45,No,No,No,5.0,No,No,"No, borderline diabetes",No,Yes,No,Fair
9,Male,65-69,White,40.69,No,No,Yes,10.0,Yes,No,No,No,No,No,Good


In [66]:
#curious to see which columns are numerical and categorical and the distribution 
numerical_cols = [col for col in dfnew.columns if dfnew.dtypes[col] != 'O']
categorical_cols = [col for col in dfnew.columns if col not in numerical_cols]

print(numerical_cols)
print(categorical_cols)

['BMI', 'SleepTime']
['Sex', 'AgeCategory', 'Race', 'Smoking', 'AlcoholDrinking', 'PhysicalActivity', 'Difficulty Walking', 'HeartDisease', 'Diabetic', 'Asthma', 'KidneyDisease', 'SkinCancer', 'GenHealth']


In [90]:
#statistical distribution of numerical categories 
dfnewest = dfnew[numerical_cols].describe()

#deleting the first row because it has no significance (had just added all the numbers together and gave total count)
dfnewest = dfnewest.drop(dfnewest.index[0])
dfnewest 


,BMI,SleepTime
mean,28.325399,7.097075
std,6.356100,1.436007
min,12.020000,1.000000
25%,24.030000,6.000000
50%,27.340000,7.000000
75%,31.420000,8.000000
max,94.850000,24.000000
